In [1]:
import numpy as np
import pandas as pd
import string
import re

In [2]:
with open('businessCardsTabs.csv', mode='r',encoding='utf8',errors='ignore') as f:
    text = f.read()

In [3]:
data= list(map(lambda x:x.split('\t'),text.split('\n')))

In [4]:
df = pd.DataFrame(data[1:],columns=data[0])

In [5]:
whitespace = string.whitespace
punctuation = '!#$%&\'()*+:;<=>?[\\]^`{|}~'

tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)

def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removeWhitespace = text.translate(tableWhitespace)
    removePunctuation = removeWhitespace.translate(tablePunctuation)
    
    return str(removePunctuation)

In [6]:
df['text'] = df['text'].apply(cleanText)

In [7]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

/home/xiaodie/.local/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [8]:
dataClean.head(10)

,id,text,tag
0,card_29.jpg,@,O
1,card_29.jpg,creative,B-ORG
2,card_29.jpg,developers,I-ORG
3,card_29.jpg,ideas,O
4,card_29.jpg,developed,O
5,card_29.jpg,by,O
6,card_29.jpg,creative,O
7,card_29.jpg,experts,O
8,card_29.jpg,kim,B-NAME
9,card_29.jpg,lou,I-NAME


In [20]:
group = dataClean.groupby(by='id')
cards = group.groups.keys()

In [16]:
grouparray = group.get_group('card_46.jpg')[['text', 'tag']].values
content = ''
annotations = {'entities': []}
start = 0
end = 0

for text, tag in grouparray:
  print(text,tag)

mike B-NAME
polinowski I-NAME
business B-DES
card I-DES
designer I-DES
987-4575-9567 B-PHONE
8365-9686-6997 B-PHONE
building O
4 O
room O
748 O
business O
plaza O
mong O
kok O
hongkong O
lookmum@iamdoingphotoshop.com B-EMAIL


In [24]:
allCardsData = []
for card in cards:
  cardData = []
  grouparray = group.get_group(card)[['text', 'tag']].values
  content = ''
  annotations = {'entities': []}
  start = 0
  end = 0

  for text, tag in grouparray:
    text = str(text)
    stringLength = len(text) + 1 # 1 x space after each word

    start = end
    end = start + stringLength

    if tag != 'O':
      annot = (start,end-1,tag)
      annotations['entities'].append(annot)

    content = content + text + ' '

  cardData = (content, annotations)
  allCardsData.append(cardData)

In [25]:
allCardsData

[('i treasare s - lcesle ‘res-ahe grade teacher el rancho elementary school 1234 anyplace street any town ca 95000 phone 408 555-1234 cell 408 555-2143 ',
  {'entities': [(31, 36, 'B-DES'),
    (37, 44, 'I-DES'),
    (45, 47, 'B-ORG'),
    (48, 54, 'I-ORG'),
    (55, 65, 'I-ORG'),
    (66, 72, 'I-ORG'),
    (118, 121, 'B-PHONE'),
    (122, 130, 'I-PHONE'),
    (136, 139, 'B-PHONE'),
    (140, 148, 'I-PHONE')]}),
 ('andrew shim ‘fre/elance programmer 012-384-5678 andrewshim@ascoders.com facebook.com/andrewshim www.ascoders.com ',
  {'entities': [(0, 6, 'B-NAME'),
    (7, 11, 'I-NAME'),
    (12, 23, 'B-DES'),
    (24, 34, 'I-DES'),
    (35, 47, 'B-PHONE'),
    (48, 71, 'B-EMAIL'),
    (72, 95, 'B-WEB'),
    (96, 112, 'B-WEB')]}),
 ('@ companylogo john smith . g s ',
  {'entities': [(14, 18, 'B-NAME'), (19, 24, 'I-NAME')]}),
 ('q ny 1234 @ @ designer ', {'entities': []}),
 ('banford i 0 123456789 ',
  {'entities': [(0, 7, 'B-NAME'), (12, 21, 'B-PHONE')]}),
 ('accountant company name your 

In [30]:
import random

random.shuffle(allCardsData)

TrainData = allCardsData[:50]
TestData = allCardsData[50:]

In [31]:
import pickle
pickle.dump(TrainData,open('./data/TrainData.pickle',mode='wb'))
pickle.dump(TestData,open('./data/TestData.pickle',mode='wb'))